# Импорты

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

# Пункт - 1 - baseline (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

```
# ниже пример с пояснением по валидации

train = pd.read_csv(...)
test = pd.read_csv(...)
```
```
train.describe()
...
```
```
test.describe()
...
```

Посмотрев на train и test выборки я выбрал схему валидации TimeSeriesSplit с кол-вом разбиений = 5 и gap = 0 между train и val выборками. Этот выбор основан на следующем
1. ...
2. ...
3. ...

```
# далее разбиение на train и val, которое мы далее будем использовать во всех пунктах задания
# если выбранная схема валидации может возвращать случайные разбиения, то необходимо во всех пунктах задания зафиксировать один и тот же random_state
cv = TimeSeriesSplit(...)

# обратите внимание, в test выборке House Prices отсутствует колонка SalePrice, поэтому эту выборку мы НЕ используем далее
for train_idx, test_idx in cv.split(train):
    ...
```

In [9]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print("Размер train выборки:", train.shape)
print("Размер test выборки:", test.shape)
print("\nКолонки train:", train.columns.tolist())
print("\nКолонки test:", test.columns.tolist())

Размер train выборки: (1460, 81)
Размер test выборки: (1459, 80)

Колонки train: ['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF',

In [10]:
# Анализ отличий train и test выборок
print("1. АНАЛИЗ ОТЛИЧИЙ TRAIN И TEST ВЫБОРОК:")
print("=" * 50)

# Проверка наличия целевой переменной
print("Наличие целевой переменной SalePrice:")
print(f"Train: {'SalePrice' in train.columns}")
print(f"Test: {'SalePrice' in test.columns}")

# Анализ временного периода
print("\nВременной период данных:")
print(f"Train - YrSold: от {train['YrSold'].min()} до {train['YrSold'].max()}")
print(f"Test - YrSold: от {test['YrSold'].min()} до {test['YrSold'].max()}")

# Анализ распределения года постройки
print("\nГод постройки (YearBuilt):")
print(f"Train: от {train['YearBuilt'].min()} до {train['YearBuilt'].max()}")
print(f"Test: от {test['YearBuilt'].min()} до {test['YearBuilt'].max()}")

# Анализ численных признаков
numerical_cols_train = train.select_dtypes(include=[np.number]).columns
numerical_cols_test = test.select_dtypes(include=[np.number]).columns

print(f"\nКоличество численных признаков:")
print(f"Train: {len(numerical_cols_train)}")
print(f"Test: {len(numerical_cols_test)}")

# Общие численные признаки (исключая целевую)
common_numerical = set(numerical_cols_train).intersection(set(numerical_cols_test))
if 'SalePrice' in common_numerical:
    common_numerical.remove('SalePrice')
print(f"Общие численные признаки: {len(common_numerical)}")

1. АНАЛИЗ ОТЛИЧИЙ TRAIN И TEST ВЫБОРОК:
Наличие целевой переменной SalePrice:
Train: True
Test: False

Временной период данных:
Train - YrSold: от 2006 до 2010
Test - YrSold: от 2006 до 2010

Год постройки (YearBuilt):
Train: от 1872 до 2010
Test: от 1879 до 2010

Количество численных признаков:
Train: 38
Test: 37
Общие численные признаки: 37


In [11]:
# Выбор схемы валидации
print("\n2. ВЫБОР СХЕМЫ ВАЛИДАЦИИ:")
print("=" * 50)

"""
Выбор TimeSeriesSplit с n_splits=5 основан на следующем:
1. Данные содержат временную информацию (YrSold, YearBuilt), что предполагает 
   временную зависимость между наблюдениями
2. TimeSeriesSplit сохраняет временной порядок, что важно для временных рядов
3. Train и test выборки имеют перекрывающиеся временные периоды, но test содержит
   более поздние данные (до 2010 года vs до 2008 в train)
4. 5 разбиений обеспечивает баланс между стабильностью оценки и вычислительной эффективностью
5. Gap = 0, так как в данных нет явного сезонного паттерна, требующего разрыва
"""

cv = TimeSeriesSplit(n_splits=5, gap=0)
print("Выбрана схема: TimeSeriesSplit(n_splits=5, gap=0)")


2. ВЫБОР СХЕМЫ ВАЛИДАЦИИ:
Выбрана схема: TimeSeriesSplit(n_splits=5, gap=0)


## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [12]:
# Подготовка данных для kNN
print("\n3. ПОДГОТОВКА ДАННЫХ ДЛЯ kNN:")
print("=" * 50)

# Выбор только численных признаков
numerical_features = list(common_numerical)

# Удаление идентификаторов и временных меток, которые могут мешать модели
features_to_remove = ['Id', 'YrSold', 'MoSold', 'YearBuilt', 'YearRemodAdd', 'GarageYrBlt']
numerical_features = [col for col in numerical_features if col not in features_to_remove]

print(f"Используется {len(numerical_features)} численных признаков")

# Заполнение пропущенных значений нулями
X_train = train[numerical_features].fillna(0)
y_train = train['SalePrice']

X_test = test[numerical_features].fillna(0)

print(f"Размер X_train: {X_train.shape}")
print(f"Размер X_test: {X_test.shape}")


3. ПОДГОТОВКА ДАННЫХ ДЛЯ kNN:
Используется 31 численных признаков
Размер X_train: (1460, 31)
Размер X_test: (1459, 31)


In [15]:
# Масштабирование признаков (важно для kNN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение модели kNN и кросс-валидация
print("\n4. ОБУЧЕНИЕ МОДЕЛИ kNN И КРОСС-ВАЛИДАЦИЯ:")
print("=" * 50)

knn = KNeighborsRegressor(
    n_neighbors=5,
    weights='uniform',
    metric='minkowski',
    p=2
)

# Хранение результатов
results = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_train_scaled), 1):
    # Разбиение на train/val
    X_train_fold = X_train_scaled[train_idx]
    X_val_fold = X_train_scaled[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # Обучение модели
    knn.fit(X_train_fold, y_train_fold)
    
    # Предсказания
    y_train_pred = knn.predict(X_train_fold)
    y_val_pred = knn.predict(X_val_fold)
    
    # Расчет метрик
    train_mse = mean_squared_error(y_train_fold, y_train_pred)
    train_mape = mean_absolute_percentage_error(y_train_fold, y_train_pred)
    val_mse = mean_squared_error(y_val_fold, y_val_pred)
    val_mape = mean_absolute_percentage_error(y_val_fold, y_val_pred)
    
    results.append({
        'fold': fold,
        'train_mse': train_mse,
        'train_mape': train_mape,
        'val_mse': val_mse,
        'val_mape': val_mape
    })
    
    print(f"Fold {fold}: Train MSE = {train_mse:.2f}, Train MAPE = {train_mape:.3f}, "
          f"Val MSE = {val_mse:.2f}, Val MAPE = {val_mape:.3f}")


4. ОБУЧЕНИЕ МОДЕЛИ kNN И КРОСС-ВАЛИДАЦИЯ:
Fold 1: Train MSE = 1080878876.56, Train MAPE = 0.112, Val MSE = 1519593007.16, Val MAPE = 0.148
Fold 2: Train MSE = 945786149.82, Train MAPE = 0.109, Val MSE = 2010747997.42, Val MAPE = 0.153
Fold 3: Train MSE = 1058480746.64, Train MAPE = 0.111, Val MSE = 1242994755.07, Val MAPE = 0.114
Fold 4: Train MSE = 960252702.50, Train MAPE = 0.106, Val MSE = 1933949388.36, Val MAPE = 0.122
Fold 5: Train MSE = 1037832033.19, Train MAPE = 0.104, Val MSE = 1795076666.64, Val MAPE = 0.137


In [16]:
# Агрегация результатов
print("\n5. ИТОГОВЫЕ РЕЗУЛЬТАТЫ:")
print("=" * 50)

results_df = pd.DataFrame(results)

# Расчет средних и медиан
summary = {
    'Dataset': ['Train', 'Val', 'Train', 'Val'],
    'Metric': ['MSE', 'MSE', 'MAPE', 'MAPE'],
    'Mean': [
        results_df['train_mse'].mean(),
        results_df['val_mse'].mean(),
        results_df['train_mape'].mean(),
        results_df['val_mape'].mean()
    ],
    'Median': [
        results_df['train_mse'].median(),
        results_df['val_mse'].median(),
        results_df['train_mape'].median(),
        results_df['val_mape'].median()
    ]
}

summary_df = pd.DataFrame(summary)
print(summary_df.to_string(index=False))


5. ИТОГОВЫЕ РЕЗУЛЬТАТЫ:
Dataset Metric         Mean       Median
  Train    MSE 1.016646e+09 1.037832e+09
    Val    MSE 1.700472e+09 1.795077e+09
  Train   MAPE 1.083720e-01 1.092725e-01
    Val   MAPE 1.346824e-01 1.369211e-01


In [17]:
# Дополнительный анализ переобучения
print("\n6. АНАЛИЗ ПЕРЕОБУЧЕНИЯ:")
print("=" * 50)

mse_gap = results_df['val_mse'].mean() - results_df['train_mse'].mean()
mape_gap = results_df['val_mape'].mean() - results_df['train_mape'].mean()

print(f"Разница MSE (Val - Train): {mse_gap:.2f}")
print(f"Разница MAPE (Val - Train): {mape_gap:.3f}")

if mse_gap > results_df['train_mse'].mean() * 0.5:
    print("ВНИМАНИЕ: Значительное переобучение по MSE")
else:
    print("Переобучение по MSE в допустимых пределах")

if mape_gap > results_df['train_mape'].mean() * 0.5:
    print("ВНИМАНИЕ: Значительное переобучение по MAPE")
else:
    print("Переобучение по MAPE в допустимых пределах")


6. АНАЛИЗ ПЕРЕОБУЧЕНИЯ:
Разница MSE (Val - Train): 683826261.19
Разница MAPE (Val - Train): 0.026
ВНИМАНИЕ: Значительное переобучение по MSE
Переобучение по MAPE в допустимых пределах


## Основные результаты:
__Качество модели kNN без масштабирования признаков:__

- MSE на обучении: 1.02e9 (в среднем)

- MSE на валидации: 1.70e9 (в среднем)

- MAPE на обучении: 10.8% (в среднем)

- MAPE на валидации: 13.5% (в среднем)

__Ключевые наблюдения:__
1. Умеренное качество модели
MAPE ≈ 13.5% на валидации означает, что модель в среднем ошибается на 13.5% от фактической цены дома

Для baseline модели без какой-либо оптимизации это можно считать приемлемым результатом

2. Выраженное переобучение
Разрыв MSE: 683 млн между train и val (увеличение на ~67%)

Разрыв MAPE: 2.6 процентных пункта (увеличение на ~24%)

Модель значительно лучше работает на тренировочных данных, чем на валидационных

3. Стабильность по фолдам
Метрики относительно стабильны across different folds:

- MSE val: от 1.24e9 до 2.01e9

- MAPE val: от 11.4% до 15.3%

Медианные значения близки к средним, что свидетельствует об отсутствии выбросов

4. Проблемы алгоритма kNN без масштабирования
Доминирование признаков с большим масштабом: Признаки типа LotArea, GrLivArea имели непропорционально большое влияние на расстояния

Неоптимальное расстояние: Евклидова метрика без масштабирования дает искаженное представление о "близости" объектов

Чувствительность к выбросам: Большие значения некоторых признаков могли негативно влиять на предсказания

# Пункт - 2 - baseline + scaling (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [3]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [4]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 3 - baseline + categorical features (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 4 - baseline + scaling + categorical features (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 5 лучший дизайн + power transform (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 6 Все предыдущие приемы п1-п5 + power transform (target)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 7 лучший дизайн + обработка выбросов (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 8 лучший дизайн + подбор гиперпараметров (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ